In [1]:
import crnindsp
import numpy as np
import pandas as pd

# Structural Decomposition
This package $\texttt{crnindsp}$ is proposed to do the structural decomposition of chemical reaction networks.
The main propose is not to search for all the buffering structures.

An advantage of this toolbox, in comparison to the package $\texttt{ibuffpy}$, is that this package is runable on singular networks.

## Example 1: a singular network

In [2]:
# Example 1: a singular network.
print("Given the stoichiometric matrix (nu)")
nu = np.array([[ -1,  0,  0,  1,  0],
               [  0, -1,  0,  0,  1],
               [ -1, -1,  2,  0,  0],
               [  1,  0, -1,  0,  0],
               [  0,  1, -1,  0,  0],], dtype = float)
print(nu)
print("without specifying the regulatory information,")
print("one can use crnindsp.CRN(nu) to construct a chemical reaction network object,")
print("and the A-matrix is given by crnindsp.CRN(nu).A as")
crn = crnindsp.CRN(nu)
A   = np.array(crn.A)
print(A)
print("By putting the CRN class object as the input of crnindsp.indicator_diagnose(),")
print("one can obtains a structural decomposition")
decomp = crnindsp.indicator_diagnose(crn)
DF = pd.DataFrame(decomp)
DF.index = DF.index[::-1] # this step is not necessary
display(DF)

Given the stoichiometric matrix (nu)
[[-1.  0.  0.  1.  0.]
 [ 0. -1.  0.  0.  1.]
 [-1. -1.  2.  0.  0.]
 [ 1.  0. -1.  0.  0.]
 [ 0.  1. -1.  0.  0.]]
without specifying the regulatory information,
one can use crnindsp.CRN(nu) to construct a chemical reaction network object,
and the A-matrix is given by crnindsp.CRN(nu).A as
[[inf  0. inf  0.  0.  1.]
 [ 0. inf inf  0.  0.  1.]
 [ 0.  0.  0. inf inf  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  1.  1.  0.]]
By putting the CRN class object as the input of crnindsp.indicator_diagnose(),
one can obtains a structural decomposition


,chemicals,reactions,sign_det
2,[X0],[R0],1.0
1,[X1],[R1],1.0
0,"[X2, X3, X4]","[R2, R3, R4]",0.0


In $\texttt{crnindsp}$, we use (numpy.inf, -numpy.inf, numpy.nan, 0) to indicate the sign patterns of qualitative regulatory information.
$$
    (\mathtt{numpy.inf}, \mathtt{-numpy.inf}, \mathtt{numpy.nan}, \mathtt{0}) = (+, -, \pm, 0)
$$
Note that, in algorithms of $\texttt{crnindsp}$, the elementary arithmetic on signs is abit different from that in $\texttt{numpy}$.

For instance, we define $0\times\text{``}\pm\text{''}=0$, but $\mathtt{0}\times\mathtt{numpy.nan}\neq\mathtt{0}$.
Hence, users are not encouraged to manipulate sign-pattern or hybrid matrices such as $\texttt{crnindsp.CRN().A}$ on their own.

## Example 2: a regular network

In [3]:
# Example 2: a regular network
print("Similarly, given a stoichiometric matrix")
S = np.array([[ -1,  1,  0,  0,  0,  0,  0,  0,],
              [  1, -1, -1,  1, -1,  0,  0,  0,],
              [  0,  0,  1, -1, -1,  3,  0,  0,],
              [  0,  0,  0,  0,  0, -3,  0,  1,],
              [  0,  0,  0,  0,  1,  0, -1,  0,],], dtype = float)
print(S)
print("and this time we further consider the regulatory information given by")
reginfo = S.copy() * 0
reginfo[S < 0] = np.inf # since reactants should regulate the reactions positively
# and we put
reginfo[0, 1]  = np.nan # when assuming that chemical X0 can regulate reaction R1 positively and negatively
reginfo[4, 7]  = np.inf # when assuming that chemical X4 regulates reaction R7 positively
# Of course, one can assign (-inf) to an entry for a negative regulation.
print(reginfo)

Similarly, given a stoichiometric matrix
[[-1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1. -1. -1.  1. -1.  0.  0.  0.]
 [ 0.  0.  1. -1. -1.  3.  0.  0.]
 [ 0.  0.  0.  0.  0. -3.  0.  1.]
 [ 0.  0.  0.  0.  1.  0. -1.  0.]]
and this time we further consider the regulatory information given by
[[inf nan  0.  0.  0.  0.  0.  0.]
 [ 0. inf inf  0. inf  0.  0.  0.]
 [ 0.  0.  0. inf inf  0.  0.  0.]
 [ 0.  0.  0.  0.  0. inf  0.  0.]
 [ 0.  0.  0.  0.  0.  0. inf inf]]


When constructing the crnindsp.CRN object, users can specify the chemical names (resp. reaction names) by the argument X_names (resp. R_names).

In [4]:
# Let's name the chemicals
chem_names = ['A', 'B', 'C', 'D', 'E']

# Then, the chemical reaction network is constructed by putting
crn5sp = crnindsp.CRN(S, reginfo, X_names = chem_names) # and R_names if you want to name the reactions.
# The A-matrix is
A_5sp  = np.array(crn5sp.A)
print("Then the A-matrix is")
print(A_5sp.round(2))

# And we can analoguously do the analysis:
decomp = crnindsp.indicator_diagnose(crn5sp)
DF5sp = pd.DataFrame(decomp)
DF5sp.index = DF5sp.index[::-1]
print("The structural decomposition is")
display(DF5sp)
# idx_eliminable = (np.sign(DF5sp.sign_det).abs() - 1).abs() < 1e-9
# print(DF5sp.loc[~idx_eliminable, :])

Then the A-matrix is
[[  inf  0.    0.    0.    0.    1.    0.    0.  ]
 [  nan   inf  0.    0.    0.    1.    0.    0.  ]
 [ 0.     inf  0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.     inf  0.    0.    0.    0.    1.  ]
 [ 0.     inf   inf  0.    0.    0.   -1.    1.  ]
 [ 0.    0.    0.     inf  0.    0.   -0.67  0.67]
 [ 0.    0.    0.    0.     inf  0.   -1.    1.  ]
 [ 0.    0.    0.    0.     inf  0.   -2.    2.  ]]
The structural decomposition is


,chemicals,reactions,sign_det
3,[D],[R5],1.0
2,[A],"[R0, R1]",NaN
1,"[B, C]","[R2, R3, R4]",-1.0
0,[E],"[R6, R7]",NaN
